In [1]:
import math
import time
import wandb
import spam
import pprint

In [2]:
#spam.mlp_classifier((784, 30, 10), epochs=1)

In [3]:
#spam.reduce_precision(1.3, 8)
dir(spam)

['__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'error',
 'matrix_factorization',
 'mlp_classifier',
 'reduce_precision',
 'system']

In [4]:
from distutils.sysconfig import get_python_lib
print(get_python_lib())

c:\users\lubko\appdata\local\programs\python\python39\Lib\site-packages


In [5]:
#import importlib
#importlib.reload(spam)

In [6]:
wandb.login()
#wandb.init(project="my-test-project")

wandb: Currently logged in as: lubomirkurcak (use `wandb login --relogin` to force relogin)


True

In [7]:
# vytvorenie sweep configu (pouzivame nahodne zvolenie parametrov)
sweep_config = {
    'method': 'random'
}

In [8]:
#metric = {
#    'name': 'loss',
#    'goal': 'minimize'
#}

metric = {
    'name': 'correct_classifications',
    'goal': 'maximize'
}

sweep_config['metric'] = metric

In [9]:
# diskretne parametre
parameters_dict = {
    'fp_subnormal': {
        'values': ['CPFLOAT_SUBN_RND',
                   'CPFLOAT_SUBN_USE']
    },
    
    'fp_round': {
        'values': ['CPFLOAT_RND_NA',
                   'CPFLOAT_RND_NZ',
                   'CPFLOAT_RND_NE',
                   'CPFLOAT_RND_TP',
                   'CPFLOAT_RND_TN',
                   'CPFLOAT_RND_TZ',
                   'CPFLOAT_RND_SP',
                   'CPFLOAT_RND_SE',
                   'CPFLOAT_RND_OD',
                   'CPFLOAT_NO_RND']
    },
    
    'fp_flip': {
        'values': ['CPFLOAT_NO_SOFTERR',
                   'CPFLOAT_SOFTERR']
    },
    
    'fp_explim': {
        'values': ['CPFLOAT_EXPRANGE_STOR',
                   'CPFLOAT_EXPRANGE_TARG']
    },
}

sweep_config['parameters'] = parameters_dict

In [10]:
# konstantne parametre
parameters_dict.update({
    'layer_sizes': {
        'value': (784, 30, 10)
    },
    'epochs': {
        'value': 5
    },
    'minibatch_size': {
        'value': 10
    },
    'learning_rate': {
        'value': 0.1
    },
    'weight_decay': {
        'value': 0.0001
    },
    'dropout': {
        'value': 0
    },
    'momentum_coefficient': {
        'value': 0
    },
    'loss': {
        'value': 'mean_squared_error'
    }
})

In [11]:
'''
precision
   * @brief Bits of precision of target format.
   *
   * @details The maximum values allowed are 24 and 53 if the storage format is
   * `float` or `double`, respectively.
'''

'''
maximum exponent
   * @brief Maximum exponent of target format.
   *
   * @details The maximum values allowed are 127 and 1023 if the storage format
   * is `float` or `double`, respectively. Larger values are reduced to the
   * maximum allowed value without warning. This field is ignored unless
   * `explim` is set to `CPFLOAT_EXPRANGE_TARG`.   
'''

# spojite parametre
parameters_dict.update({
    'fp_p': {
        'distribution': 'uniform',
        'min': 0,
        'max': 0.5
    },
    
    'fp_precision': {
        'distribution': 'q_uniform',
        'q': 1,
        'min': 4,
        'max': 24,
    },
    
    'fp_emax': {
        'distribution': 'q_uniform',
        'q': 1,
        'min': 4,
        'max': 127,
    },
})


'''
'fp_precision': {
    'distribution': 'q_log_uniform',
    'q': 1,
    'min': math.log(4),
    'max': math.log(24),
},

'fp_emax': {
    'distribution': 'q_log_uniform',
    'q': 1,
    'min': math.log(4),
    'max': math.log(127),
},
'''

"\n'fp_precision': {\n    'distribution': 'q_log_uniform',\n    'q': 1,\n    'min': math.log(4),\n    'max': math.log(24),\n},\n\n'fp_emax': {\n    'distribution': 'q_log_uniform',\n    'q': 1,\n    'min': math.log(4),\n    'max': math.log(127),\n},\n"

In [12]:
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'correct_classifications'},
 'parameters': {'dropout': {'value': 0},
                'epochs': {'value': 5},
                'fp_emax': {'distribution': 'q_uniform',
                            'max': 127,
                            'min': 4,
                            'q': 1},
                'fp_explim': {'values': ['CPFLOAT_EXPRANGE_STOR',
                                         'CPFLOAT_EXPRANGE_TARG']},
                'fp_flip': {'values': ['CPFLOAT_NO_SOFTERR',
                                       'CPFLOAT_SOFTERR']},
                'fp_p': {'distribution': 'uniform', 'max': 0.5, 'min': 0},
                'fp_precision': {'distribution': 'q_uniform',
                                 'max': 24,
                                 'min': 4,
                                 'q': 1},
                'fp_round': {'values': ['CPFLOAT_RND_NA',
                                        'CPFLOAT_RND_NZ',
                     

In [13]:
spam.system('echo %TIME%')

0

In [14]:
# inicializuj sweep pomocou konfigu (vytvori sa stranka sweepu, kde je mozne monitorovat behy a analyzovat parametre)
sweep_id = wandb.sweep(sweep_config, project="my-test-project")

Create sweep with ID: fc3gl8vd
Sweep URL: https://wandb.ai/lubomirkurcak/my-test-project/sweeps/fc3gl8vd


In [15]:
# trenovacia funkcia, pouzita pri volani trenovacich agentov
def train():
    
    # inicialuzuj wandb a vyber config s nastavenymi hodnotami podla sweep configu
    run = wandb.init()
    config = run.config
    
    '''
    for dummy_epoch in range(30):
        wandb.log({"loss": 0.9 ** dummy_epoch,
                   "accuracy": 1 - 0.97 ** dummy_epoch}, commit=False, step=dummy_epoch)
    '''
    
    correct_classifications = spam.mlp_classifier(tuple(config["layer_sizes"]),
        epochs=config["epochs"],
        minibatch_size=config["minibatch_size"],
        learning_rate=config["learning_rate"],
        weight_decay=config["weight_decay"],
        dropout=config["dropout"],
        momentum_coefficient=config["momentum_coefficient"],
        fp_precision=config["fp_precision"],
        fp_emax=config["fp_emax"],
        fp_subnormal=config["fp_subnormal"],
        fp_round=config["fp_round"],
        fp_flip=config["fp_flip"],
        fp_p=config["fp_p"],
        fp_explim=config["fp_explim"]
    )
    
    
    wandb.log({'correct_classifications': correct_classifications}, commit=False, step=0)
    
    '''
    print('Early accuracy stop!')
    wandb.alert(
        title='Early accuracy stop!', 
        text=f'Accuracy {correct/total} is above 0.7',
        level=AlertLevel.INFO, # INFO, WARN, or ERROR 
        wait_duration=timedelta(minutes=1)
    )
    '''
        
    run.finish()
    

In [16]:
#spam.create_forward_net([784, 30, 10])
#spam.create_forward_net((784, 30, 10))

In [17]:
#wandb.agent("i91zir54", project="my-test-project", function=train, count=400)

In [ ]:
wandb.agent(sweep_id, train, count=4)

wandb: Agent Starting Run: ymrljv8s with config:
wandb: 	dropout: 0
wandb: 	epochs: 5
wandb: 	fp_emax: 18
wandb: 	fp_explim: CPFLOAT_EXPRANGE_TARG
wandb: 	fp_flip: CPFLOAT_SOFTERR
wandb: 	fp_p: 0.13106019853764334
wandb: 	fp_precision: 10
wandb: 	fp_round: CPFLOAT_RND_SP
wandb: 	fp_subnormal: CPFLOAT_SUBN_USE
wandb: 	layer_sizes: [784, 30, 10]
wandb: 	learning_rate: 0.1
wandb: 	loss: mean_squared_error
wandb: 	minibatch_size: 10
wandb: 	momentum_coefficient: 0
wandb: 	weight_decay: 0.0001
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


cpfloat_validate_optstruct returned 0
Epoch 0/5 Batch 0/6000 Cost 1.920241 LR 0.100000
Epoch 0/5 Batch 1024/6000 Cost 0.341838 LR 0.100000
Epoch 0/5 Batch 2048/6000 Cost 0.121723 LR 0.100000
Epoch 0/5 Batch 3072/6000 Cost 0.330341 LR 0.100000
Epoch 0/5 Batch 4096/6000 Cost 0.257503 LR 0.100000
Epoch 0/5 Batch 5120/6000 Cost 0.254673 LR 0.100000
Epoch 0/5 Correct 8968/10000 Best 8968(0)Epoch 1/5 Batch 0/6000 Cost 0.097667 LR 0.100000
Epoch 1/5 Batch 1024/6000 Cost 0.351005 LR 0.100000
Epoch 1/5 Batch 2048/6000 Cost 0.106928 LR 0.100000
Epoch 1/5 Batch 3072/6000 Cost 0.331145 LR 0.100000
Epoch 1/5 Batch 4096/6000 Cost 0.231015 LR 0.100000
Epoch 1/5 Batch 5120/6000 Cost 0.249076 LR 0.100000
Epoch 1/5 Correct 8998/10000 Best 8998(0)Epoch 2/5 Batch 0/6000 Cost 0.096376 LR 0.100000
Epoch 2/5 Batch 1024/6000 Cost 0.368299 LR 0.100000
Epoch 2/5 Batch 2048/6000 Cost 0.097662 LR 0.100000
Epoch 2/5 Batch 3072/6000 Cost 0.323273 LR 0.100000
Epoch 2/5 Batch 4096/6000 Cost 0.231447 LR 0.100000
Epoch

wandb: Network error (SSLError), entering retry loop. See W:\rpnn\python\wandb\run-20220301_225944-ymrljv8s\logs\debug-internal.log for full traceback.


Epoch 4/5 Batch 5120/6000 Cost 0.181177 LR 0.100000
Epoch 4/5 Correct 9088/10000 Best 9088(0)

correct_classifications,9088
_runtime,1459
_timestamp,1646173443
_step,0


...

Epoch 3/5 Correct 9256/10000 Best 9256(0)Epoch 4/5 Batch 0/6000 Cost 0.072502 LR 0.100000
Epoch 4/5 Batch 1024/6000 Cost 0.347712 LR 0.100000
Epoch 4/5 Batch 2048/6000 Cost 0.062326 LR 0.100000
Epoch 4/5 Batch 3072/6000 Cost 0.291297 LR 0.100000
Epoch 4/5 Batch 4096/6000 Cost 0.156302 LR 0.100000
Epoch 4/5 Batch 5120/6000 Cost 0.158451 LR 0.100000
Epoch 4/5 Correct 9275/10000 Best 9275(0)

wandb: Network error resolved after 0:11:01.184327, resuming normal operation.


correct_classifications,9275
_runtime,685
_timestamp,1646174396
_step,0


correct_classifications,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Agent Starting Run: l0zj4p80 with config:
wandb: 	dropout: 0
wandb: 	epochs: 5
wandb: 	fp_emax: 33
wandb: 	fp_explim: CPFLOAT_EXPRANGE_STOR
wandb: 	fp_flip: CPFLOAT_SOFTERR
wandb: 	fp_p: 0.26739102312733076
wandb: 	fp_precision: 17
wandb: 	fp_round: CPFLOAT_RND_SP
wandb: 	fp_subnormal: CPFLOAT_SUBN_RND
wandb: 	layer_sizes: [784, 30, 10]
wandb: 	learning_rate: 0.1
wandb: 	loss: mean_squared_error
wandb: 	minibatch_size: 10
wandb: 	momentum_coefficient: 0
wandb: 	weight_decay: 0.0001
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [ ]:
'''
import gym
env = gym.make('CartPole-v0')

for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        env.render()
        #print(observation)
        #action = env.action_space.sample()
        #action = (t+1)//2%2
        action = 1 if observation[2] > 0 else 0

        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()
'''